In [71]:
import torch
import torch.nn as nn

import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import visdom

vis=visdom.Visdom()

Setting up a new session...


In [72]:
vis.text('hello world!',env='main')             # env='main'은 있어도 되고 없어도 된다. 
                                                # 이건 어떨 때 쓰냐면 한번에 모든 창을 끌 때 사용할 수 있다.
                                                #  한번에 main에 있는 것을 다 끄고 싶을 때 모든 vis 관련된 생성 창들에 env를 넣어주는 것이 중요하다.

'window_3dcfdbb63554fa'

In [73]:
a=torch.randn(3,200,200)        #3으로 하면 rgb로 된다. 200x200. random으로 생성해라
vis.image(a)

'window_3dcfdbb638c9a2'

In [74]:
# 이미지를 여러개 띄우는 방법
vis.images(torch.Tensor(3,3,28,28))         #28x28의 이미지가 3개 생성된 것을 볼 수 있다.

'window_3dcfdbb63a4c7a'

In [75]:
# mnist와 cifar10이라는 데이터셋을 가져와보자.
MNIST=dsets.MNIST(root='C:/git_experiment/RWL_Intern/dh_workspace/모두를 위한 딥러닝/MNIST',
                        train=True,
                        transform=transforms.ToTensor(),
                        download=True)
cifar10=dsets.CIFAR10(root='C:/git_experiment/RWL_Intern/dh_workspace/모두를 위한 딥러닝/CIFAR10',
                        train=True,
                        transform=transforms.ToTensor(),
                        download=True)


In [76]:
data=cifar10.__getitem__(0)             #첫번째 데이터를 가져오고 
print(data[0].shape)                    #그 데이터의 shape값을 확인
vis.images(data[0],env='main')

torch.Size([3, 32, 32])


'window_3dcfdbb6a82178'

In [77]:
data=MNIST.__getitem__(0)
print(data[0].shape)
vis.images(data[0],env='main')

torch.Size([1, 28, 28])


'window_3dcfdbb6a9f4de'

In [78]:
#DataLoader를 이용해 여러개의 이미지를 볼 수 있는 방법도 있다.
data_loader=torch.utils.data.DataLoader(dataset=MNIST,batch_size=32, shuffle=True)

In [79]:
for num, value in enumerate(data_loader):
    value=value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


In [90]:
vis.close(env='main')           # env='main'인 애들을 모두 꺼주라는 얘기

''

# LINE PLOT

In [81]:
Y_data=torch.randn(5)               
plt=vis.line(Y=Y_data)          # 점 다섯개가 생긴다. 만약 X값을 넣지 않으면 X가 0~1사이의 값에서 Y의 점이 찍힌다.

In [82]:
X_data=torch.Tensor([1,2,3,4,5])
plt=vis.line(X=X_data, Y=Y_data)

# LINE UPDATE

In [83]:
# 예를 들어 loss값을 하나씩 추가할 때.
# 들어가야할 x값과 들어가야할 loss값을 정의를 해서 line에다가 넣어주면 된다.
Y_append=torch.randn(1)
X_append=torch.Tensor([6])

vis.line(X=X_append,Y=Y_append, win=plt, update='append')

'window_3dcfdbb6b1818e'

# Multiple Lines on a Single Window

In [84]:
num=torch.Tensor(list(range(0,10)))
print(num)
print(num.shape)
num=num.view(-1,1)
print(num)
print(num.shape)
num=torch.cat((num,num), dim=1)             #(10,2) 형태로 바꾼 이유는 아래의 Y의 형태와 맞춰주려고(line을 두개 그리기 위해서.)
print(num)
print(num.shape)
plt=vis.line(Y=torch.randn(10,2),X=num)

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])
torch.Size([10])
tensor([[0.],
        [1.],
        [2.],
        [3.],
        [4.],
        [5.],
        [6.],
        [7.],
        [8.],
        [9.]])
torch.Size([10, 1])
tensor([[0., 0.],
        [1., 1.],
        [2., 2.],
        [3., 3.],
        [4., 4.],
        [5., 5.],
        [6., 6.],
        [7., 7.],
        [8., 8.],
        [9., 9.]])
torch.Size([10, 2])


# Line Info

In [91]:
plt=vis.line(Y=Y_data,X=X_data,opts=dict(title='Test',showlegend=True))     # opts= 을 하고 dictionary 형태로 넣어준다. 

In [ ]:
# 축의 이름 자체를 설정하는 방법이 있다.
plt=vis.line(Y=Y_data, X=X_data, opts=dict(title='Test',legend=['1번'],showlegend=True))

In [94]:
# 선이 두개인 경우
plt=vis.line(Y=torch.randn(10,2), X=num, opts=dict(title='Test', legend=['1번','2번'],showlegend=True))

# Make function for update line

In [ ]:
# cnn에 visdom을 넣어서 loss function이 update 되는 것을 plot해보자.

def loss_tracker(loss_plot, loss_value, num):       # update할 plot이름, loss 값, x값 
    '''num, loss_value, and Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win=loss_plot,
             update='append')

In [101]:
plt=vis.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss=torch.randn(1)+i
    loss_tracker(plt,loss,torch.Tensor([i]))

# close the window

In [102]:
vis.close(env='main')

''

# Mnist를 이용해서 실습을 해보자.

In [103]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms 
import torch.nn.init

import visdom

vis=visdom.Visdom()
vis.close(env='main')


Setting up a new session...


''

In [104]:
# cnn에 visdom을 넣어서 loss function이 update 되는 것을 plot해보자.

def loss_tracker(loss_plot, loss_value, num):       # update할 plot이름, loss 값, x값 
    '''num, loss_value, and Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win=loss_plot,
             update='append')

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [106]:
# parameters
learning_rate=0.001
training_epochs=15
batch_size=100

# MNIST dataset
mnist_train=dsets.MNIST(root='C:/git_experiment/RWL_Intern/dh_workspace/모두를 위한 딥러닝/MNIST',
                        train=True,
                        transform=transforms.ToTensor(),
                        download=True)      #transform은 input data를 어떻게 변환해줄 거냐를 의미함. 
                                            #그냥 받아오면 Tensor value가 아니므로받아온 mnist data를 tensor value로 바꿔준다.
mnist_test=dsets.MNIST(root='C:/git_experiment/RWL_Intern/dh_workspace/모두를 위한 딥러닝/MNIST',
                       train=False,
                       transform=transforms.ToTensor(),
                       download=True)

In [107]:
data_loader=torch.utils.data.DataLoader(dataset=mnist_train,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        drop_last=True)

In [108]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()                  #  CNN,self는 현재 클래스(여기서는 CNN)와 현재 객체(여기서는 self)를 명시적으로 지정하는 
        self.layer1=torch.nn.Sequential(
            torch.nn.Conv2d(1,32,kernel_size=3,stride=1, padding=1),
            torch.nn.ReLU(),
            nn.MaxPool2d(2)                         # MaxPool2d를 거쳐도 채널의 개수는 변하지 않는다.
        )

        self.layer2=torch.nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc=nn.Linear(7*7*64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(out)

        out=out.view(out.size(0),-1)
        out=self.fc(out)
        return out

In [109]:
model=CNN().to(device)

criterion=nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

Let's make a plot for mnist

In [110]:
loss_plt=vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker',legend=['loss'],showlegend=True))

training with loss_tracker

In [ ]:
#training 하는 코드
total_batch=len(data_loader)                # 전체 배치 수가 나온다.
for epoch in range(training_epochs):
    avg_cost=0

    for X,Y in data_loader:
        X=X.to(device)                      # cuda 연산을 진행하려면 해야한다.
        Y=Y.to(device)                      # cuda 연산을 진행하려면 해야한다.

        optimizer.zero_grad()
        hypothesis=model(X)
        cost=criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()

        avg_cost+=cost/total_batch

    print('epoch: {} cost: {}'.format(epoch+1,avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]),torch.Tensor([epoch]))      # epoch 마다 avg_cost를 업데이트를 시킨다.

print('learning finished')

epoch: 1 cost: 0.22577044367790222
epoch: 2 cost: 0.06312766671180725
epoch: 3 cost: 0.04643740877509117
epoch: 4 cost: 0.03755507618188858
epoch: 5 cost: 0.03148474171757698
epoch: 6 cost: 0.026334717869758606
epoch: 7 cost: 0.021956756711006165
epoch: 8 cost: 0.018336869776248932
epoch: 9 cost: 0.016243524849414825
epoch: 10 cost: 0.013560776598751545
epoch: 11 cost: 0.010381104424595833
epoch: 12 cost: 0.009994762018322945
epoch: 13 cost: 0.008455739356577396
epoch: 14 cost: 0.006931321229785681
epoch: 15 cost: 0.006881385575979948
learning finished
